# 📘 Piranha Ground Truth Data Generator
Dieses Notebook erzeugt Piranha-kompatible Ground Truth Daten: Aus der Manuell Gelabelten Excel wird eine JSON Datei erzeugt, die die Labels mit Ihrer Position erfasst. Inputs sind das golden_dataset_original und die Excel /data/DAiA Manual Labeling.xlsx, Output ist dann /data/piranha_training_data.json

In [1]:
import os
import re
import json
import pandas as pd
from pathlib import Path

GROUPED_PLACEHOLDERS = {
    "TITEL": ["TITEL"],
    "VORNAME": ["VORNAME"],
    "NACHNAHME": ["NACHNAME", "SKYPE"],
    "ADRESSE": ["ADRESSE"],
    "VERTRAGSNUMMER": ["VERTRAGSNUMMER"],
    "KUNDENNUMMER": ["KUNDENNUMMER", "ZUORDNUNGSNUMMER", "WLV"],
    "BETRAG": ["ZAHLUNG"],
    "IBAN": ["IBAN"],
    "BIC": ["BIC"],
    "ZÄHLERSTAND": ["ZÄHLERSTAND"],
    "ZÄHLERNUMMER": ["ZÄHLERNUMMER"],
    "VERBRAUCH": ["VERBRAUCH"],
    "TELEFONNUMMER": ["TELEFONNUMMER"],
    "EMAIL": ["EMAIL", "MAIL"],
    "LINK": ["LINK"],
    "GESENDET_MIT": ["GESENDET_MIT"],
    "FAX": ["FAX"],
    "FIRMA": ["FIRMA"],
    "DATUM": ["DATUM"]
}

def map_column_to_label(col_name):
    upper_col = col_name.upper()
    for label, keywords in GROUPED_PLACEHOLDERS.items():
        for keyword in keywords:
            if keyword in upper_col:
                return label
    return None

def generate_piranha_training_data(df, email_folder_path, output_file_path):
    training_data = []

    for index, row in df.iterrows():
        file_name = row["TextFile"]
        email_path = os.path.join(email_folder_path, file_name)

        try:
            with open(email_path, "r", encoding="utf-8") as file:
                text = file.read()
        except FileNotFoundError:
            print(f"Datei nicht gefunden: {file_name}")
            continue

        labels = []
        for col in row.index:
            if pd.isna(row[col]):
                continue
            label_type = map_column_to_label(col)
            if label_type:
                value = str(row[col]).strip()
                start_idx = text.lower().find(value.lower())
                if start_idx != -1:
                    end_idx = start_idx + len(value)
                    labels.append({"start": start_idx, "end": end_idx, "label": label_type})
                else:
                    print(f"Wert nicht gefunden: '{value}' in Datei {file_name}")

        training_data.append({
            "text": text,
            "labels": labels
        })

    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(training_data, f, ensure_ascii=False, indent=2)

    print(f"Piranha-Trainingsdaten gespeichert unter: {output_file_path}")


In [3]:
# 📥 Excel laden und Funktion ausführen
df = pd.read_excel("../../data/DAiA_Manual_Labeling_zusammengefasst.xlsx", dtype=str)
generate_piranha_training_data(
    df,
    "../data/golden_dataset_original_copy",
    "../data/piranha_ground_truth.json"
)

Piranha-Trainingsdaten gespeichert unter: ../data/piranha_ground_truth.json
